<a href="https://colab.research.google.com/github/zerotodeeplearning/ztdl-masterclasses/blob/master/notebooks/Model_Evaluation_and_Dimensionality_Reduction_with_Scikit_Learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Copyright 2020 Catalit LLC.

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Model Evaluation and Dimensionality Reduction with Scikit Learn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
url = 'https://raw.githubusercontent.com/zerotodeeplearning/ztdl-masterclasses/master/data/'

In [ ]:
df = pd.read_csv(url + 'sms.tsv', sep='\t')

In [ ]:
df.head()

In [ ]:
df['label'].value_counts() / len(df)

In [ ]:
y = (df['label'] == 'spam')

### Word count features

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
def cross_val_score_print(model, X, y, cv=3):
    scores = cross_val_score(model, X, y, cv=cv)
    print("Accuracy score: {:0.3} +/- {:0.3}".format(scores.mean(), scores.std()))

In [ ]:
vect = CountVectorizer(decode_error='ignore',
                       stop_words='english',
                       binary=True,
                       max_features=2000)

X = vect.fit_transform(df['msg'])

In [ ]:
X

Visualize the first 200 word counts in the first 200 messages

In [ ]:
N = 200
plt.figure(figsize=(10, 10))
plt.imshow(X.todense()[:N, :N]);

In [ ]:
cross_val_score_print(DummyClassifier(strategy='most_frequent'), X, y)

In [ ]:
cross_val_score_print(LogisticRegression(solver='liblinear'), X, y)

### Feature importances

The model using 2000 word features seems to be performing quite well. Let's find out which words are more correlated with `Spam`.

The features we are using are counts of word occurrences in a corpus of SMS messages. Since SMSs have a fixed length, we can assume that these counts are proportional to the fequencies of occurrences. In other words we can assume that all the features have the same normalization scale. Under this assumption, we can interpret the coefficients of the `LogisticRegression` model as features importances.

In [ ]:
model = LogisticRegression(solver='liblinear')
model.fit(X, y)

In [ ]:
word_feature_importances = pd.Series(model.coef_[0],
                                     index=vect.get_feature_names()).sort_values()

In [ ]:
# Top 20 least spammy words
word_feature_importances.head(20)

In [ ]:
# Top 20 most spammy words
word_feature_importances.tail(20)

### Truncated SVD

A common way to visualize highly dimensional feature sets is to use the Truncated SVD dimensionality reduction technique. Let's use it to compress our 2000 sparse features to a 5 dimensional space.

In [ ]:
from sklearn.decomposition import TruncatedSVD

In [ ]:
X_tsvd = pd.DataFrame(TruncatedSVD(n_components=5).fit_transform(X), columns=['c1', 'c2', 'c3', 'c4', 'c5'])
X_tsvd['label'] = y

In [ ]:
sns.pairplot(X_tsvd, hue='label');

### Exercise 1: Model validation with Pipelines.

When the feature engineering step involves learning something from the data we should only learn from the training set.

In the case above, we are learning the vocabulary from the data, but there are many other cases where the transformer is learning  properties of the data. In these cases, we should proceed with caution and only learn from the training set.

One way to achieve this is to do something like this:

```python
raw_features_train, raw_features_test = train_test_split(....)
transformer = ....

transformer.fit(raw_features_train)
X_train = transformer.transform(raw_features_train)
X_test = transformer.transform(raw_features_test)
```

a better way to achieve the same is to bundle the transformer and the estimator into a [`Pipeline`](https://scikit-learn.org/stable/modules/compose.html).

Complete the following steps:

- Split `df['msg']` and `y` into train and test sets
- Create a pipeline using the `make_pipeline` function that contains at least 2 steps: `vect` and `LogisticRegression()`. Feel free to include additional intermediate steps if you wish
- Train the pipeline model on the trainin set and compare the training and test scores
- Bonus points if you perform Cross Validation


In [ ]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from sklearn.pipeline import make_pipeline

### Exercise 2: ROC curve and learning curve

- Use the trained pipeline model to calculate the [`roc_curve`](https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html)
- Bonus point if you plot it for both train and test sets defined above.
- Use the pipeline model to calculate and plot the [`learning_curve`](https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html)

### Exercise 3

Let's explore the effect of Dimensionality Reduction techniques on a different dataset: the Digits dataset.

The data is loaded for you.

- Use one or more dimensionality reduction techniques (e.g. `PCA`, `TSNE` or other) to compress the 64 pixel features into 2 features.
- Use `sns.scatterplot` to visualize the whole dataset in the reduced space
- Use the `y` variable to color the data: do you see clusters of similar points appear?

In [ ]:
from sklearn.datasets import load_digits

In [ ]:
X, y = load_digits(return_X_y=True)

In [ ]:
X.shape

In [ ]:
plt.imshow(X[0].reshape(8, 8), cmap='gray');

In [ ]:
plt.imshow(X[1].reshape(8, 8), cmap='gray');